# Review Text analysis _ Translator()

# 데이터 준비하기

In [1]:
# 가져오기 전에 라이브러리 불러오기
# pd로 불러올 것이기 때문에 pandas도 미리 필요하다
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime as dt

In [2]:
import glob
paths = glob.glob("data/*.*")
paths

['data/olist_sellers_dataset.csv',
 'data/product_category_name_translation.csv',
 'data/olist_orders_dataset.csv',
 'data/olist_order_items_dataset.csv',
 'data/olist_customers_dataset.csv',
 'data/olist_geolocation_dataset.csv',
 'data/olist_order_payments_dataset.csv',
 'data/olist_order_reviews_dataset.csv',
 'data/olist_products_dataset.csv']

In [3]:
df_sellers = pd.read_csv('data/olist_sellers_dataset.csv')
df_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_order_items = pd.read_csv('data/olist_order_items_dataset.csv')
df_customers = pd.read_csv('data/olist_customers_dataset.csv')
df_geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
df_order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
df_order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
df_products = pd.read_csv('data/olist_products_dataset.csv')
df_product_category_name_translation = pd.read_csv('data/product_category_name_translation.csv')

In [4]:
df_orders.describe().T

,count,unique,top,freq
order_id,99441,99441,e481f51cbdc54678b7cc49136f2d6af7,1
customer_id,99441,99441,9ef432eb6251297304e76186b10a928d,1
order_status,99441,8,delivered,96478
order_purchase_timestamp,99441,98875,2018-04-11 10:48:14,3
order_approved_at,99281,90733,2018-02-27 04:31:10,9
order_delivered_carrier_date,97658,81018,2018-05-09 15:48:00,47
order_delivered_customer_date,96476,95664,2018-05-08 23:38:46,3
order_estimated_delivery_date,99441,459,2017-12-20 00:00:00,522


In [5]:
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])
df_orders['order_approved_at'] = pd.to_datetime(df_orders['order_approved_at'])
df_orders['order_delivered_carrier_date'] = pd.to_datetime(df_orders['order_delivered_carrier_date'])
df_orders['order_delivered_customer_date'] = pd.to_datetime(df_orders['order_delivered_customer_date'])
df_orders['order_estimated_delivery_date'] = pd.to_datetime(df_orders['order_estimated_delivery_date'])

In [6]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [7]:
df_reviews = pd.merge(df_orders, df_order_reviews, left_on='order_id', right_on='order_id')
df_reviews.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03 00:00:00,2017-12-05 19:21:58
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 13:02:51


In [8]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99224 non-null  object        
 1   customer_id                    99224 non-null  object        
 2   order_status                   99224 non-null  object        
 3   order_purchase_timestamp       99224 non-null  datetime64[ns]
 4   order_approved_at              99068 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97468 non-null  datetime64[ns]
 6   order_delivered_customer_date  96359 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99224 non-null  datetime64[ns]
 8   review_id                      99224 non-null  object        
 9   review_score                   99224 non-null  int64         
 10  review_comment_title           11568 non-null  object        
 11  review_comment_

In [9]:
df_reviews["review_creation_date"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews["review_answer_timestamp"] = pd.to_datetime(df_reviews["review_creation_date"])
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99224 non-null  object        
 1   customer_id                    99224 non-null  object        
 2   order_status                   99224 non-null  object        
 3   order_purchase_timestamp       99224 non-null  datetime64[ns]
 4   order_approved_at              99068 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97468 non-null  datetime64[ns]
 6   order_delivered_customer_date  96359 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99224 non-null  datetime64[ns]
 8   review_id                      99224 non-null  object        
 9   review_score                   99224 non-null  int64         
 10  review_comment_title           11568 non-null  object        
 11  review_comment_

In [10]:
df_reviews.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-11
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18,2018-08-18
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,359d03e676b3c069f62cadba8dd3f6e8,5,NaN,O produto foi exatamente o que eu esperava e e...,2017-12-03,2017-12-03
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,e50934924e227544ba8246aeb3770dd4,5,NaN,NaN,2018-02-17,2018-02-17


In [11]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99224 entries, 0 to 99223
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99224 non-null  object        
 1   customer_id                    99224 non-null  object        
 2   order_status                   99224 non-null  object        
 3   order_purchase_timestamp       99224 non-null  datetime64[ns]
 4   order_approved_at              99068 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97468 non-null  datetime64[ns]
 6   order_delivered_customer_date  96359 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99224 non-null  datetime64[ns]
 8   review_id                      99224 non-null  object        
 9   review_score                   99224 non-null  int64         
 10  review_comment_title           11568 non-null  object        
 11  review_comment_

# 리뷰 데이터 EDA

## 리뷰데이터 번역하기 Translator()

In [12]:
# EDA 전 시각화 데이터 출력 디자인 세팅
# 윈도우 : "Malgun Gothic"
# 맥 : "AppleGothic"

def get_font_family():
    """
    시스템 환경에 따른 기본 폰트명을 반환하는 함수
    """
    import platform
    system_name = platform.system()
    
    if system_name == "Darwin" :
        font_family = "AppleGothic"
    elif system_name == "Windows" :
        font_family ="Malgun Gothic"
    else:
        !apt-get install fonts-nanum -qq  > /dev/null
        !fc-cache -fv
        
        import matplotlib as mpl
        mpl.font_manager._rebuild()
        findfont = mpl.font_manager.fontManager.findfont
        mpl.font_manager.findfont = findfont
        mpl.backends.backend_agg.findfont = findfont
        
        font_family = "NanumBarunGothic"
    return font_family


plt.style.use("seaborn-darkgrid")

#폰트설정
plt.rc("font", family = get_font_family())
#마이너스폰트 설정
plt.rc("axes", unicode_minus = False)

#그래프에 retina display 적용
from IPython.display import set_matplotlib_formats

set_matplotlib_formats("retina")

/var/folders/3q/y966c1fj3lndmps8kk3plpy40000gn/T/ipykernel_50861/1557497376.py:40: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("retina")


In [14]:
from googletrans import Translator

In [15]:
# translator.translate( )로 사용하고,
# translator.translate(번역할 단어)
# 결과값 src="번역전언어", dest = "번역후언어", text= 번역된 언어 (주로 영어로 세팅되어 있음)
# # translator.translate(번역할 단어).text하면 결과만 나옴
from googletrans import Translator
translator = Translator()

In [16]:
translator.translate("Muito bom o produto.").text
#dest가 번역후 언어인데, 기본으로 english가 세팅되어있음

'The product is very good.'

In [17]:
translator.translate("Muito bom o produto.", dest = "ko").text

'제품은 매우 좋습니다.'

## 1 리뷰 스코어 별로, 리뷰 메세지 가져오기

In [173]:
s_m = df_reviews[['review_score','review_comment_message']]

- na값이 아닌 순수 텍스트만 보고자하므로 dropna 진행

In [ ]:
sm = s_m.dropna()

In [180]:
sm

,review_score,review_comment_message
0,4,"Não testei o produto ainda, mas ele veio corre..."
1,4,Muito bom o produto.
3,5,O produto foi exatamente o que eu esperava e e...
6,2,fiquei triste por n ter me atendido.
10,1,Aguardando retorno da loja
...,...,...
99214,5,Otimo comprar por este site as mercadorias che...
99215,1,"Ele não é um mini cajon, é um shaker, ou seja,..."
99220,4,So uma peça que veio rachado mas tudo bem rs
99221,5,Foi entregue antes do prazo.


In [182]:
for i in range(len(sm['review_comment_message'])):
    sm['review_comment_message'][i] = sm['review_comment_message'].to_list()[i].replace('\r\n','')
sm

/var/folders/3q/y966c1fj3lndmps8kk3plpy40000gn/T/ipykernel_50861/2089022670.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sm['review_comment_message'][i] = sm['review_comment_message'].to_list()[i].replace('\r\n','')
/Users/jeon-eunji/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1724: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer_missing(indexer, value)


,review_score,review_comment_message
0,4,"Não testei o produto ainda, mas ele veio corre..."
1,4,Muito bom o produto.
3,5,fiquei triste por n ter me atendido.
6,2,Obrigado pela atenção. Lojas lannister perfeit...
10,1,Até o momento não recebi o produto Protetor De...
...,...,...
99214,5,Otimo comprar por este site as mercadorias che...
99215,1,"Ele não é um mini cajon, é um shaker, ou seja,..."
99220,4,So uma peça que veio rachado mas tudo bem rs
99221,5,Foi entregue antes do prazo.


r01 = [0] * len(Rscore01['review_comment_message'])
for i in range(len(Rscore01['review_comment_message'])):
    r01[i] = Rscore01['review_comment_message'].to_list()[i].replace('\r\n','')
r01 
r = pd.DataFrame(r01, columns=["review_comment_message"])
r

In [183]:
Rscore01 = sm[sm['review_score'] == 1]

In [184]:
len(Rscore01['review_comment_message'])

8745

In [185]:
Rscore02 = sm[sm['review_score'] == 2]
Rscore03 = sm[sm['review_score'] == 3]
Rscore04 = sm[sm['review_score'] == 4]
Rscore05 = sm[sm['review_score'] == 5]

In [186]:
pd.options.display.max_rows = 100

In [187]:
Rscore05.head(100)

,review_score,review_comment_message
3,5,fiquei triste por n ter me atendido.
15,5,há muito tempo efetuo compras atraves desta lo...
23,5,"Inicialmente, pedi o cancelamento do pedido de..."
26,5,"Pedir 2 capinhas,estou pagando as duas e só ve..."
32,5,O produto é uma ótima decoração para deixar o ...
34,5,Muito bom o produto e preço cobrado!
37,5,"Os outros produtos, a loja ofereceu mas não cu..."
39,5,Recebi 22 dias antes do prazo
47,5,Espero que seja sempre assim rápido e produto ...
53,5,O suporte veio com um parafuso espanado... Com...


In [190]:
Rscore01.shape, Rscore02.shape, Rscore03.shape, Rscore04.shape, Rscore05.shape

((8745, 2), (2145, 2), (3557, 2), (5976, 2), (20554, 2))

In [192]:
#갑자기 궁금한 올리스트 전체 리뷰 평균
(8745 * 1 + 2145 *2 + 3557 * 3 + 5976 * 4 + 20554 * 5) / 40977

3.66986358200942

In [223]:
translator.translate(Rscore01['review_comment_message'].to_list()[0]).text

"So far I have not received the protective product of crib mattress - waterproof cotton, I've made the complaint since 18/06 and has not yet gotten return.I will cancel the order and reversal the purchase hj"

In [221]:
Rscore01.head(5)

,review_score,review_comment_message
10,1,Até o momento não recebi o produto Protetor De...
25,1,Entrega antes da data marcada. Excelente
30,1,Agora é testar para avaliar se é de qualidade....
40,1,"Produto muito bonito, chegou no prazo correto ..."
44,1,Não recebi o produto até a presente data na mi...


In [ ]:
#리뷰테이블에서 리뷰스토어와 리뷰텍스트만 가져온다 
review = df_reviews[['review_score','review_comment_message']]
#리뷰 점수 별로 서브셋을 지정한다
score01 = review[review['review_score'] == 1]
score02 = review[review['review_score'] == 2]
score03 = review[review['review_score'] == 3]
score04 = review[review['review_score'] == 4]
score05 = review[review['review_score'] == 5]

In [224]:
score01['review_comment_message'].to_list()[3]

'Produto muito bonito, chegou no prazo correto parece um Trofeu, qualidade ótimo para presente'

In [217]:
#포트루갈어는 알파벳 위에 성조? 같은 문자들을 특수문자로 표현해서 이걸 except 하는 for문 실행 
translations = []
for i in range(len(score02['review_comment_message'])):
    try: 
        translations.append(translator.translate(score02['review_comment_message'].to_list()[i]).text)
    except:
        translations.append('E_R_R_O_R')
translations

["So far I have not received the protective product of crib mattress - waterproof cotton, I've made the complaint since 18/06 and has not yet gotten return.I will cancel the order and reversal the purchase hj",
 'Delivery before the scheduled date.Great',
 'Now is testing to evaluate if it is of quality.But so far all right.',
 'Very beautiful product, it has arrived on the right time it looks like a trophy, great quality for gift',
 'I did not receive the product to date at my residence.',
 'Recommended',
 'I think I should send the products together, as in the invoice we were together and are the same luminaries and not send almost a week after difference.',
 'Product does not meet expectations',
 'I requested the return of the order! Due to making another purchase, but everything came right in the other order!',
 'Super recommend, it is exactly as I thought, arrived 9 days before.Congratulations Baratheon.',
 'I am very satisfied',
 'Practical!',
 'Super fast delivery.Product is goo

In [216]:
translator.translate(Rscore01['review_comment_message'].to_list()[8743]).text

'Shelves with vivo corners, columns do not fit, there are lamelasses so as not to damage the painting, shelves arrived all wrinkled.'

In [215]:
Rscore01['review_comment_message'].to_list()[8743]

'Prateleiras com cantos vivo, colunas nao encaixam, faltam arroelas para nao danificar a pintura, prateleiras chegaram todas amassadas.'

- 너무 시간이 오래걸려서 테이블당 sample(frac = 0.1) 정도로 추출해서 분석진행할 수 밖에 없었음

In [232]:
R01S = Rscore01.sample(frac = 0.1, random_state = 42)
R01S

,review_score,review_comment_message
12001,1,Muito bonito!!!! Satisfeita com o material!!!
67349,1,Meu cadastro estava incorreto e só vi após a c...
60507,1,A compra foi de 2 produtos e até o momento ape...
48087,1,produto diferente do anunciado
61642,1,Produto não foi entregue
...,...,...
1328,1,"Adorei a compra, entrega super rápida."
16091,1,Boneco de ótima qualidade. Entrega muito rápid...
3129,1,"ACHEI TUDO PÉSSIMO, NÃO TEM IDENTIFICAÇÃO NA E..."
67277,1,Um dos relogios veio aberto sem tampa e sem fu...


In [233]:
translations = []
for i in range(len(R01S['review_comment_message'])):
    try: 
        translations.append(translator.translate(R01S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations.append('E_R_R_O_R')
translations

['아주 귀엽다 !!!!재료에 만족 !!!',
 '등록이 잘못되었고 주문 확인 후에 만 \u200b\u200b보았습니다.시스템에서 등록을 변경했으며 27/02/18 이후 웹 사이트 및 전화를 통해 연락했습니다.지금까지 나는이 요청을받지 못했습니다. 3 개의 카트리지',
 '구매는 2 개의 제품이었고 지금까지 18/12 년에 내 거주지에 도달 한 것 중 1 명에 불과합니다. 송장에 2 개의 제품이 포함되어 있기 때문에 다른 제품에 대한 수익을 기다리고 있습니다.',
 '발표 된 다른 제품',
 '제품은 배달되지 않았습니다',
 '매우 얇은 카펫 ... 아름답지만 바닥은 더 두껍습니다.',
 '제품이 정확했지만 지침 매뉴얼이 누락되었고 회사에 연락했지만 답을 얻지 못했습니다. 사용 방법에 대한 참조가 없기 때문에 장비가 중지되었습니다!',
 '마감일 전에 배달되었습니다 .... 아주 좋은 자료',
 '제 시간에 배달, 모두 OK !!',
 '생체 내 모서리가있는 선반, 기둥은 맞지 않으며, 그림을 손상시키지 않도록 라멜라스가 있으며, 선반은 모두 주름진 곳에 도착했습니다.',
 '제 시간에 전달되었지만 제품은 완전히 잘못되었습니다.나는 전화 Xiaomi Redmi Note 4x에 대한 표지와 영화를 요청했습니다.그리고 다른 큰 전화에서 덮개와 영화가 나왔습니다.',
 '훌륭한',
 '재고가없는 제품을 제시했습니다',
 '제 시간에 제품을 받았지만 깨졌고 이미 이메일을 보냈으며 무슨 일이 있었는지 알리는 두 번째 평가입니다.나는 대답을 기다리고있다.',
 '우체국 때문인지는 모르겠지만, 사실은 상점 주인이 불만을 제기 한 후에도 저에게 연락하지 않았기 때문에 제품을받을 것인지조차 알지 못한다는 것입니다.',
 '끔찍한 한 달이 걸렸고 그곳에서 제품을 보내지 않았고 오늘날까지 돈을 반환하지 않았습니다.',
 '제품 판매량은 오리지널 Oakley 브랜드가 아니며 복제품이며 소비자에게는 완전히 무례합니다!',
 '아직 테스트하지 않았습니다.',
 '지금까지 제품을받지 못

In [234]:
R02S = Rscore02.sample(frac = 0.1, random_state = 42)
R03S = Rscore03.sample(frac = 0.1, random_state = 42)
R04S = Rscore04.sample(frac = 0.1, random_state = 42)
R05S = Rscore05.sample(frac = 0.1, random_state = 42)

In [239]:
translations2 = []
for i in range(len(R02S['review_comment_message'])):
    try: 
        translations2.append(translator.translate(R02S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations2.append('E_R_R_O_R')
translations2

['나는 우편물에 가야했다.나는 나의 거주지로 배달하기 위해 배송을 지불했다!',
 '배달은 우체국에 의한 것이되어서는 안됩니다.\r\n 그래서 나에게 그것은 매우 나쁘다.',
 '좋아했습니다',
 '고기 연삭 기계 만 받았는데 두 번을 샀는데 왜 두 번째로 오지 않았는가?내가 구입 한 카트도 도착하지 않았습니다',
 '나는 양호한 것을 발견했다.',
 '제품은 철수를 기다리고 있었고 전화로 통보되지 않았습니다.네트 슈즈와 포스트가 항상 나에게 주면 왜 배달하지 않기 때문에 옳은 일은 집에서받는 것입니다.이거 이해 못 하겠어요.',
 '매우 빠른 축하였습니다',
 '품질이 좋지',
 '트리플 엑스',
 '빠른 배송 및 우수한 제품.설치하기 쉽습니다',
 '버릇없는 유리와 함께 제공되었습니다',
 '표지는 괜찮 았지만 유리 영화가 잘못되었습니다 .... 더 작아요 ....',
 '나는 2 개의 램프를 샀고 하나만왔다',
 '..',
 '상점이 제품을 배달하는 데 너무 오래 걸렸고화물이 비싸다고 생각했습니다.',
 '나는 단 하나의 제품 만 받았고 2 개를 주문했다.\r\n다른 제품을 기다리고 있습니다.\r\n내가받은 제품은 양질의 제품입니다.',
 '단일 사이트를 통해 모든 것을 구매하여 배송비를 지불하는 것은 단순히 끔찍합니다.나는 또 다른 상점이지만 Lannister.com에서 다시 구매하지는 않습니다.',
 '구매하려고하는 시간이 다르고 제품이 잘못되었습니다.그들은 재고가없는 제품, 두통 만있는 제품을 판매합니다. 권장하지 않습니다.',
 '전달 된 제품은 광고 된 제품이 아닙니다.나는 반환 과정에있다.',
 '수분을 흡수하는 나무로 만들어졌습니다 ..\n나는 권장하지 않는다.사용 된 재료에 대해 매우 비쌉니다.',
 '고객에 대한 존중 부족',
 '발표 된 제품에 따르면 제품이 마음에 들었습니다',
 '내가 그녀에게 전화하자마자 그녀는 터졌다.제품 교환을 요청합니다',
 '나는 팔콘을 사서 개미 사람을 받았다.',
 '납치',
 '지금까지 나는 마지막 쿠

In [240]:
translations3 = []
for i in range(len(R03S['review_comment_message'])):
    try: 
        translations3.append(translator.translate(R03S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations3.append('E_R_R_O_R')
translations3

['내 제품이 오늘까지 전달되지 않았다는 사실을 알고 싶습니다.',
 '팀은 축하를 받아야한다 !!!',
 '나는 모두에게 추천합니다',
 '품질이 좋지 않은 영화',
 '아주 좋은 서비스와 정보가 요청되었습니다.',
 '나는 수건이 매우 못 생겼다는 것을 알았습니다. 사진에서 그들은 흰색으로 보였지만 얼음과 품질이 훨씬 낮습니다.',
 '좋은 아침! 나는 의자를 받았지만 오래 가지 못했지만 약혼자는 낚시에 사용되어 팔의지지를 부러졌습니다.그는 82 파운드가 철을 부러 뜨리지 않았다.다른 것을 보낼 것인지 확인하십시오',
 '훌륭한',
 '이 경우 컴퓨터 소스였던 내 제품은 전달되지 않았습니다.그러나 그들은 훌륭한 서비스를 가지고 있습니다.',
 '배달 시간과 모든 것이 완벽한 주문으로 나왔습니다.유일한 문제는 마감일이 실제로 걸린다는 것입니다.도착하는 데 약 14 일이 걸렸습니다.이 외에도 판매와 관련된 모든 사람들에게 10을 알 수 있습니다.',
 '훌륭한 재료',
 '나는 더 기대했다!',
 '아직 문제가 없었습니다',
 '그에 따라 서비스',
 '제품이 약간 시원해 졌기 때문에 포장에는 한 가지 문제가있었습니다.',
 '매우 낮은 품질의 제품, 직물이 매우 얇은 것은? Felpuda와 아무 관련이 없습니다.제 생각에는 20.00에 팔렸다면 여전히 비싸고 매우 나쁘게 만들어졌습니다 !!!',
 '빠르게 배달하십시오!',
 '나는 이미이 사이트에서 다른 구매를했지만 이번에는 배송 시간에 만족하지 않았고 시간이 많이 걸렸습니다.\r\n안녕하세요',
 '마감일 이내에 확인되었습니다.',
 '아직받지 못했습니다',
 '나는 제 시간에 받았다',
 '제품은 매우 심하게 포장되었습니다.관리 나 보호없이 배달 상자 안에만 배치 된 시계였습니다.제품에 약간의 손상이 발생할 수 있습니다.',
 'SAC는 쓰레기입니다',
 '나는 두 개의 컨트롤을 샀고 1 만 나왔습니다',
 '나는 유리 영화에 걸리는 걸쇠를 가져 가면서 괜찮아졌지만, 그것이 누가 잘못인지 모르기 때문에 

In [241]:
translations4 = []
for i in range(len(R04S['review_comment_message'])):
    try: 
        translations4.append(translator.translate(R04S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations4.append('E_R_R_O_R')
translations4

['제 시간에 도착했고 발표 된대로!',
 '괜찮은.게시물은 스크리닝을 업데이트하는 데 속도가 느리지 만 정시에 도착했습니다.',
 '정말 즐거웠습니다',
 '마감일 전에 배달 ... 완벽한 상태의 모든 것 ... 나는 그것을 좋아했습니다!',
 '전달 및 제품 품질도 모두 괜찮습니다',
 '제품은 사진과 동일하지 않으며 배달은 합의 된대로 만들어지지 않았으며 대행사에서 제거해야합니다.대부분은 괜찮습니다.',
 '보습이 잘되지만 많이 똑 바르지 않습니다',
 '빠른 배송 및 아주 좋은 제품.',
 '내 기대를 초과했습니다',
 'BOM',
 '마감일 이전에 제공되는 제품과 매우 좋은 품질은 추천합니다.',
 '아주 좋은 서비스와 마감일 준수',
 '정시에 도착, 양질의 제품.',
 '많은 서비스, 추천 할 수 있습니다 ..',
 '나는 항상이 회사에서 운송 업체가 제품을 구매하는 제품을 얻습니다. 항상 매우 빠르지 만 우편으로 보낼 때 더 길고 제거해야합니다.',
 'BOM',
 '훌륭한 제품',
 '좋은 제품과 우수한 시간 배달',
 '제품 제품이 필요한 사람들에게 추천합니다.',
 '그것을 가져 갔지만 도착했습니다',
 '설명대로 마감일 전과 아주 좋은 제품 !!',
 '제품은 기대를 충족합니다.',
 'Lannister Delivery에서 구매하십시오',
 '제품의 배송 시간은 이미 이겼으며 공급 업체로부터 수익이 없으며 더 이상 추적 할 수 없습니다.',
 '제 시간에 전달되는 제품과 제품 품질은 매우 좋아 보입니다!',
 '제 시간에 전달되는 제품.',
 '제품이 예상되기 전에 잘 도착했고, 병이 명백한 문제없이 도착했습니다. 이제 병이 좋은지 알아 내기 위해 사용 테스트를 수행 할 것입니다.',
 '우수한 제품',
 '제품은 마감일 전에 배달되었습니다.그리고 승인되었습니다!',
 '나는 01 UN을 받았다.02/19 NF는 02 대가 있다고 진술했다.나는 Lannister에게 불만을 보냈다. 그것은 매우 불쾌했다.20 일에 나는 나머지를 받았다. 나

In [243]:
translations5 = []
for i in range(len(R05S['review_comment_message'])):
    try: 
        translations5.append(translator.translate(R05S['review_comment_message'].to_list()[i], dest = 'ko').text)
    except:
        translations5.append('E_R_R_O_R')
translations5

['Lannister 웹 사이트에서 제품을 반환하기 위해 옵션을 사용하려고했는데 오류가 발생하고 판매자에게 연락하여 제품 교환을 기다리고 있습니다.',
 '멋진 가게 !!!!',
 '시간 엄수 판매자와 제품은 저의 기대를 만족 시켰습니다.',
 '완벽한 조건에서 마감일 전에 매우 빠른 배송이 잘 도착했습니다. 완벽한 마감재는 훌륭한 품질의 Bacly 보증 제품으로 잘 포장되어 있습니다.',
 '아주 좋습니다. 마감일은 구매시 전송 된대로 충족되었습니다.',
 '합법적인',
 '단순하고 아름답고 기능적인 베개.좋아했습니다!',
 '제 시간에 배달!모든 것이 괜찮습니다.',
 '두 부분으로 부러진 제품, 나는 이미 Stark 의이 "파트너"보증을 트리거했지만 연락처를 반환하지는 않습니다!나쁜 사업.',
 '우수한 제품.\r\n판매자 노트 1000',
 '예상 날짜 이전에 완벽한 조건으로 배달.',
 '우체국은 주문을 전달하는 데 너무 오래 걸리고 있습니다.',
 '훌륭한.',
 '참으로 아주 좋습니다!',
 '이 제품은 완벽한 훌륭한 포장이 왔습니다. 다시 살 것입니다. 감사합니다',
 '배낭이 매우 간단합니다.',
 '가게에 아주 좋았습니다. 나는 결코 문제가 없었습니다.',
 '우수한 서비스와 제품이 매우 빠르게 도착했습니다.\r\n나는 가게를 추천한다.',
 '완벽한',
 '나는 스타크 쇼핑을 두려워한다고 고백한다. 나는 기뻐한다. 나는 모든 친구들을위한 스타크 웹 사이트를 추천 할 것이다 ...',
 '품질과 가격이 좋은 자료를 추천합니다.',
 '매우 빠른 배송 및 매우 잘 포장 된 제품!',
 '예상 날짜 이전에 배달.',
 '제품은 예상 마감일 전에 도착했습니다',
 '우수한 상점, 나중에 시계를 구매할 것입니다. 매장의 제공으로 무료 배송을 보낼 수 있습니다.감사해요',
 '빠르고 부드러운 배송.',
 '우수한 서비스 품질과 제품',
 '예쁜!!!마감일 전에 받았습니다.\r\n그것을 사랑.',
 '미리 전달, 훌륭한 품질',
 '이것은 Stark의 

In [247]:
pd.DataFrame(translations5)

,0
0,Lannister 웹 사이트에서 제품을 반환하기 위해 옵션을 사용하려고했는데 오류가...
1,멋진 가게 !!!!
2,시간 엄수 판매자와 제품은 저의 기대를 만족 시켰습니다.
3,완벽한 조건에서 마감일 전에 매우 빠른 배송이 잘 도착했습니다. 완벽한 마감재는 훌...
4,아주 좋습니다. 마감일은 구매시 전송 된대로 충족되었습니다.
...,...
2050,"추천, 모든 것이 잘되었습니다"
2051,매우 빠르고 안전합니다
2052,훌륭한 가게.구매에 기뻤습니다. 배송은 예상보다 일찍 도착했습니다.
2053,실용적이고 작고 가볍고 나는 거의 테스트했지만 지금은 그것을 많이 즐기고 있습니다.


- translations, translations(2,3,4,5) csv파일로 변환후 태블로에서 작업

In [263]:
translations01 = pd.DataFrame(translations)
translations02 = pd.DataFrame(translations2)
translations03 = pd.DataFrame(translations3)
translations04 = pd.DataFrame(translations4)
translations05 = pd.DataFrame(translations5)

In [264]:
# csv 파일로 저장합니다.
translations01.to_csv("Review01_translations.csv", index=False, encoding="utf-8")
translations02.to_csv("Review02_translations.csv", index=False, encoding="utf-8")
translations03.to_csv("Review03_translations.csv", index=False, encoding="utf-8")
translations04.to_csv("Review04_translations.csv", index=False, encoding="utf-8")
translations05.to_csv("Review05_translations.csv", index=False, encoding="utf-8")

In [265]:
pd.read_csv("Review05_translations.csv", encoding="utf-8")

,0
0,Lannister 웹 사이트에서 제품을 반환하기 위해 옵션을 사용하려고했는데 오류가...
1,멋진 가게 !!!!
2,시간 엄수 판매자와 제품은 저의 기대를 만족 시켰습니다.
3,완벽한 조건에서 마감일 전에 매우 빠른 배송이 잘 도착했습니다. 완벽한 마감재는 훌...
4,아주 좋습니다. 마감일은 구매시 전송 된대로 충족되었습니다.
...,...
2050,"추천, 모든 것이 잘되었습니다"
2051,매우 빠르고 안전합니다
2052,훌륭한 가게.구매에 기뻤습니다. 배송은 예상보다 일찍 도착했습니다.
2053,실용적이고 작고 가볍고 나는 거의 테스트했지만 지금은 그것을 많이 즐기고 있습니다.
